In [1]:
using CSV, DataFrames, Pkg, StatsBase, StatsPlots, Dates, Statistics

In [2]:
raw_df = CSV.read("bike_ped_counts.csv", DataFrame; stringtype=String)
total_counts = combine(
    groupby(raw_df, [:bp_loc_id, :count_type, :count_date]),
    :cnt_total => sum => :loc_total
)

┌ Warning: thread = 1 warning: parsed expected 73 columns, but didn't reach end of line around data row: 1. Parsing extra columns and widening final columnset
└ @ CSV C:\Users\maxsu\.julia\packages\CSV\XLcqT\src\file.jl:593


Row,bp_loc_id,count_type,count_date,loc_total
,Int64,String,Date,Int64
1,20016,B,1974-03-05,478
2,20016,B,1974-03-08,1001
3,20014,B,1975-10-09,904
4,10127,B,1975-10-09,376
5,10528,B,1975-10-09,213
6,10432,B,1975-10-09,167
7,20016,B,1975-10-09,420
8,341,B,1975-10-09,813
9,20030,B,1975-10-09,109


In [3]:
# Give a day_of_week column
total_counts.day_of_week = dayname.(total_counts.count_date)

4222-element Vector{String}:
 "Tuesday"
 "Friday"
 "Thursday"
 "Thursday"
 "Thursday"
 "Thursday"
 "Thursday"
 "Thursday"
 "Thursday"
 "Thursday"
 "Thursday"
 "Thursday"
 "Thursday"
 ⋮
 "Thursday"
 "Thursday"
 "Thursday"
 "Thursday"
 "Thursday"
 "Sunday"
 "Sunday"
 "Sunday"
 "Sunday"
 "Sunday"
 "Sunday"
 "Sunday"

In [5]:
# Now, we only care about types P (pedestrian), J (jogger), C (child)
relevant_counts = total_counts[[x in ["P", "J", "C"] for x in total_counts.count_type],:]

Row,bp_loc_id,count_type,count_date,loc_total,day_of_week
,Int64,String,Date,Int64,String
1,20106,P,1978-07-06,507,Thursday
2,20106,P,1978-07-13,51,Thursday
3,20106,J,1978-07-13,23,Thursday
4,20106,P,1980-06-27,72,Friday
5,20106,J,1980-06-27,28,Friday
6,20106,P,1981-08-06,164,Thursday
7,20008,P,1993-09-28,81,Tuesday
8,20013,P,1993-09-28,58,Tuesday
9,20054,P,1993-09-28,59,Tuesday


In [9]:
# Want to see which locations have P,J,C and each combination
num_all_three = 0
num_total = 0
for loc_id in unique(relevant_counts.bp_loc_id)
    has_all_three = true
    specific_counts = relevant_counts[relevant_counts.bp_loc_id .== loc_id,:]
    for mode in ["P","J","C"]
        if size(specific_counts[specific_counts.count_type .== mode,:],1) === 0
            has_all_three = false
        end
    end
    if has_all_three
        num_all_three += 1
    end
    num_total += 1
end

num_all_three, num_total

(110, 153)

In [15]:
# Isolate to just when the P,J,C are all there, for simplicity, first don't worry about day of the week
chosen_locs = []
for loc_id in unique(relevant_counts.bp_loc_id)
    has_all_three = true
    specific_counts = relevant_counts[relevant_counts.bp_loc_id .== loc_id,:]
    for mode in ["P","J","C"]
        if size(specific_counts[specific_counts.count_type .== mode,:],1) === 0
            has_all_three = false
        end
    end
    if has_all_three
        push!(chosen_locs, loc_id)
    end
end

# Now that we have the locations, create a dataframe
average_count_est = DataFrame(bp_loc_id = chosen_locs)
average_count_est.avg_count = zeros(length(chosen_locs))
for day in ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
    average_count_est[!, Symbol(day * " count")] = zeros(length(chosen_locs))
end

for (i,loc_id) in enumerate(chosen_locs)
    specific_counts = relevant_counts[relevant_counts.bp_loc_id .== loc_id,:]
    ped_counts = specific_counts[specific_counts.count_type .== "P",:]
    jog_counts = specific_counts[specific_counts.count_type .== "J",:]
    chi_counts = specific_counts[specific_counts.count_type .== "C",:]
    
    total_est = mean(ped_counts.loc_total) + 2 * mean(jog_counts.loc_total) + 0.5 * mean(chi_counts.loc_total)
    average_count_est[i, :avg_count] = total_est
    for day in ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
        average_count_est[i, Symbol(day * " count")] = total_est
    end

    # For the weekday counts, take the average if there is no corresponding data for that mode of transport and that day
    for day in ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
        dow_ped_counts = ped_counts[ped_counts.day_of_week .== day,:]
        dow_jog_counts = jog_counts[jog_counts.day_of_week .== day,:]
        dow_chi_counts = chi_counts[chi_counts.day_of_week .== day,:]

        dow_ped_mean = mean(ped_counts.loc_total)
        if size(dow_ped_counts,1) > 0
            dow_ped_mean = mean(dow_ped_counts.loc_total)
        end

        dow_jog_mean = mean(jog_counts.loc_total)
        if size(dow_jog_counts,1) > 0
            dow_jog_mean = mean(dow_jog_counts.loc_total)
        end

        dow_chi_mean = mean(chi_counts.loc_total)
        if size(dow_chi_counts,1) > 0
            dow_chi_mean = mean(dow_chi_counts.loc_total)
        end

        average_count_est[i, Symbol(day * " count")] = dow_ped_mean + 2 * dow_jog_mean + 0.5 * dow_chi_mean

        # if size(dow_ped_counts,1) > 0 && size(dow_jog_counts,1) > 0 && size(dow_chi_counts) > 0
        #     average_count_est[i, Symbol(day * " count")] = (
        #         mean(dow_ped_counts.loc_total) + 2 * mean(jog_counts.loc_total) + 0.5 * mean(chi_counts.loc_total)
    end
end

In [16]:
average_count_est

Row,bp_loc_id,avg_count,Monday count,Tuesday count,Wednesday count,Thursday count,Friday count,Saturday count,Sunday count
,Any,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,20106,301.458,301.458,315.583,233.25,287.917,129.25,301.458,301.458
2,20008,62.6667,62.6667,104.167,52.75,62.6667,62.6667,41.0,62.6667
3,20055,101.583,199.5,50.5,101.583,101.583,101.583,101.583,101.583
4,20056,115.75,115.75,115.75,115.75,115.75,115.75,115.75,115.75
5,20058,721.041,226.5,371.292,520.75,398.25,1408.0,1004.2,935.0
6,10713,81.5,81.5,81.5,81.5,81.5,81.5,81.5,81.5
7,20059,186.0,186.0,208.0,186.0,186.0,186.0,124.0,232.5
8,20003,954.926,954.926,1114.17,1005.0,588.0,1393.5,954.926,185.583
9,20120,2828.69,2828.69,1436.62,2822.25,2209.5,2828.69,3716.4,2828.69


In [17]:
average_count_est[average_count_est.avg_count .> 1000,:]

Row,bp_loc_id,avg_count,Monday count,Tuesday count,Wednesday count,Thursday count,Friday count,Saturday count,Sunday count
,Any,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,20120,2828.69,2828.69,1436.62,2822.25,2209.5,2828.69,3716.4,2828.69
2,20112,4944.0,4944.0,4944.0,4944.0,4944.0,4944.0,4944.0,4944.0
3,20113,2028.5,2028.5,2028.5,2028.5,2028.5,2028.5,2028.5,2028.5
4,20116,2906.5,2906.5,2906.5,2906.5,2906.5,2906.5,2906.5,2906.5
5,20118,3460.0,3460.0,3460.0,3460.0,3460.0,3460.0,3460.0,3460.0
6,20040,2165.33,2165.33,824.667,1533.5,2165.33,2165.33,3096.1,2165.33
7,20033,1064.57,1064.57,1268.0,1247.0,1064.57,1064.57,917.25,1064.57
8,20017,1582.5,1582.5,1582.5,1582.5,1582.5,1582.5,1582.5,1582.5
9,20145,1326.03,908.5,929.125,1326.03,1321.0,2377.25,1617.75,1326.03


In [18]:
CSV.write("mean_counts.csv", average_count_est)

"mean_counts.csv"